####################################################################################################

Team Members : Pulkit Varshney, Pratik Thorwe, Purva Makarand Mhasakar, Saurabh Parekh

Code File Description : This code uses tensorflow approach to train a Unet model based on the Semantic segmentation of aerial imagery dataset. Once the model is trained, we use to predict the segmentation on the satellite images of the Multi-Temporal Urban Development SpaceNet Dataset.

####################################################################################################

Code base has been referred and built on top of : 

Source :
1. https://towardsdatascience.com/u-net-for-semantic-segmentation-on-unbalanced-aerial-imagery-3474fa1d3e56

2. https://github.com/amirhosseinh77/UNet-AerialSegmentation

In [ ]:
import cv2
import os
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import PIL
import random
from scipy import ndimage
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
import math
from glob import glob
import sys
import shutil  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

In [19]:
class segmentClasses(torch.utils.data.Dataset):
    def __init__(self, root, training, transform=None):
        super(segmentClasses, self).__init__()
        self.root = root
        self.training = training
        self.transform = transform
        self.IMG_NAMES = sorted(glob(self.root + '/*/images/*.jpg'))
        self.BGR_classes = {'Water' : [ 41, 169, 226],
                            'Land' : [246,  41, 132],
                            'Road' : [228, 193, 110],
                            'Building' : [152,  16,  60], 
                            'Vegetation' : [ 58, 221, 254],
                            'Unlabeled' : [155, 155, 155]} # in BGR

        self.bin_classes = ['Water', 'Land', 'Road', 'Building', 'Vegetation', 'Unlabeled']


    def __getitem__(self, idx):
        img_path = self.IMG_NAMES[idx]
        mask_path = img_path.replace('images', 'masks').replace('.jpg', '.png')

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path)
        cls_mask = np.zeros(mask.shape)  
        cls_mask[mask == self.BGR_classes['Water']] = self.bin_classes.index('Water')
        cls_mask[mask == self.BGR_classes['Land']] = self.bin_classes.index('Land')
        cls_mask[mask == self.BGR_classes['Road']] = self.bin_classes.index('Road')
        cls_mask[mask == self.BGR_classes['Building']] = self.bin_classes.index('Building')
        cls_mask[mask == self.BGR_classes['Vegetation']] = self.bin_classes.index('Vegetation')
        cls_mask[mask == self.BGR_classes['Unlabeled']] = self.bin_classes.index('Unlabeled')
        cls_mask = cls_mask[:,:,0] 

        if self.training==True:
            if self.transform:
              image = transforms.functional.to_pil_image(image)
              image = self.transform(image)
              image = np.array(image)

        image = cv2.resize(image, (512,512))/255.0
        cls_mask = cv2.resize(cls_mask, (512,512)) 
        image = np.moveaxis(image, -1, 0)

        return torch.tensor(image).float(), torch.tensor(cls_mask, dtype=torch.int64)


    def __len__(self):
        return len(self.IMG_NAMES)

In [20]:
color_shift = transforms.ColorJitter(.1,.1,.1,.1)
blurriness = transforms.GaussianBlur(3, sigma=(0.1, 2.0))
t = transforms.Compose([color_shift, blurriness])
dataset = segmentClasses('./Semantic segmentation dataset', training = True, transform= t)

In [21]:
test_num = int(0.1 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset)-test_num, test_num], generator=torch.Generator().manual_seed(101))

In [22]:
BACH_SIZE = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BACH_SIZE, shuffle=True, num_workers=4)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=BACH_SIZE, shuffle=False, num_workers=1)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Source :
1. https://towardsdatascience.com/u-net-for-semantic-segmentation-on-unbalanced-aerial-imagery-3474fa1d3e56

2. https://github.com/amirhosseinh77/UNet-AerialSegmentation

In [23]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [24]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [25]:
class mIoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True, n_classes=2):
        super(mIoULoss, self).__init__()
        self.classes = n_classes

    def to_one_hot(self, tensor):
        n,h,w = tensor.size()
        one_hot = torch.zeros(n,self.classes,h,w).to(tensor.device).scatter_(1,tensor.view(n,1,h,w),1)
        return one_hot

    def forward(self, inputs, target):
        # inputs => N x Classes x H x W
        # target_oneHot => N x Classes x H x W
        N = inputs.size()[0]
        # predicted probabilities for each pixel along channel
        inputs = F.softmax(inputs,dim=1)
        # Numerator Product
        target_oneHot = self.to_one_hot(target)
        inter = inputs * target_oneHot
        ## Sum over all pixels N x C x H x W => N x C
        inter = inter.view(N,self.classes,-1).sum(2)
        #Denominator 
        union= inputs + target_oneHot - (inputs*target_oneHot)
        ## Sum over all pixels N x C x H x W => N x C
        union = union.view(N,self.classes,-1).sum(2)
        loss = inter/union
        ## Return average loss over classes and batch
        return 1-loss.mean()

In [26]:
criterion = mIoULoss(n_classes=6).to(device)

In [27]:
def acc(label, predicted):
  seg_acc = (y.cpu() == torch.argmax(pred_mask, axis=1).cpu()).sum() / torch.numel(y.cpu())
  return seg_acc

In [28]:
min_loss = torch.tensor(float('inf'))

model = UNet(n_channels=3, n_classes=6, bilinear=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

In [29]:
os.makedirs('./saved_models', exist_ok=True)

N_EPOCHS = 30
N_DATA = len(train_dataset)
N_TEST = len(test_dataset)

plot_losses = []
scheduler_counter = 0

for epoch in range(N_EPOCHS):
  # training
  model.train()
  loss_list = []
  acc_list = []
  for batch_i, (x, y) in enumerate(train_dataloader):

      pred_mask = model(x.to(device))  
      loss = criterion(pred_mask, y.to(device))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_list.append(loss.cpu().detach().numpy())
      acc_list.append(acc(y,pred_mask).numpy())

  scheduler_counter += 1
  # testing
  model.eval()
  val_loss_list = []
  val_acc_list = []
  for batch_i, (x, y) in enumerate(test_dataloader):
      with torch.no_grad():    
          pred_mask = model(x.to(device))  
      val_loss = criterion(pred_mask, y.to(device))
      val_loss_list.append(val_loss.cpu().detach().numpy())
      val_acc_list.append(acc(y,pred_mask).numpy())
    
  print(' epoch {} - loss : {:.5f} - acc : {:.2f} - val loss : {:.5f} - val acc : {:.2f}'.format(epoch, 
                                                                                                 np.mean(loss_list), 
                                                                                                 np.mean(acc_list), 
                                                                                                 np.mean(val_loss_list),
                                                                                                 np.mean(val_acc_list)))
  plot_losses.append([epoch, np.mean(loss_list), np.mean(val_loss_list)])

  compare_loss = np.mean(val_loss_list)
  is_best = compare_loss < min_loss
  if is_best == True:
    scheduler_counter = 0
    min_loss = min(compare_loss, min_loss)
    torch.save(model.state_dict(), './saved_models/unet_epoch_{}_{:.5f}.pt'.format(epoch,np.mean(val_loss_list)))
  
  if scheduler_counter > 5:
    lr_scheduler.step()
    print(f"lowering learning rate to {optimizer.param_groups[0]['lr']}")
    scheduler_counter = 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


 epoch 0 - loss : 0.84988 - acc : 0.57 - val loss : 0.87128 - val acc : 0.56
 epoch 1 - loss : 0.78913 - acc : 0.66 - val loss : 0.85466 - val acc : 0.52
 epoch 2 - loss : 0.76463 - acc : 0.67 - val loss : 0.76258 - val acc : 0.68
 epoch 3 - loss : 0.74589 - acc : 0.67 - val loss : 0.80282 - val acc : 0.62
 epoch 4 - loss : 0.73538 - acc : 0.68 - val loss : 0.73529 - val acc : 0.70
 epoch 5 - loss : 0.74284 - acc : 0.66 - val loss : 0.73029 - val acc : 0.66
 epoch 6 - loss : 0.71064 - acc : 0.70 - val loss : 0.69829 - val acc : 0.74
 epoch 7 - loss : 0.70885 - acc : 0.68 - val loss : 0.71434 - val acc : 0.72
 epoch 8 - loss : 0.70090 - acc : 0.71 - val loss : 0.71199 - val acc : 0.72
 epoch 9 - loss : 0.70510 - acc : 0.71 - val loss : 0.72309 - val acc : 0.71
 epoch 10 - loss : 0.70406 - acc : 0.69 - val loss : 0.69447 - val acc : 0.74
 epoch 11 - loss : 0.68878 - acc : 0.72 - val loss : 0.83339 - val acc : 0.30
 epoch 12 - loss : 0.70709 - acc : 0.70 - val loss : 0.69533 - val acc : 0

In [ ]:
model.load_state_dict(torch.load('/content/saved_models/unet_epoch_27_0.65264.pt'))

######################################################################################################

On the trained model, satellite images of SpaceNet dataset
are passed for prediction. Based on the output segmented graph,
land area and green area percentage coverage is obtained. The outputs are written to csv files.
#######################################################################################################

In [ ]:
import cv2 as cv
import matplotlib.image as img
from google.colab.patches import cv2_imshow
import csv
count=0
data=[]
for filename in os.listdir('/content/satelliteImgs/images/'):
        # print(filename)
        fn='/content/satelliteImgs/images/'+filename
        test_img = cv2.imread(fn)
        test_img = cv2.resize(test_img, (256,256))/255.0
        test_img = np.moveaxis(test_img, -1, 0).reshape(-1,3,256,256)
        test_img = torch.tensor(test_img).float()
        result = model(test_img.to(device))
        mask = torch.argmax(result, axis=1).cpu().detach().numpy()[0] 
        count=count+1
        fName='/content/masked'+str(count)+'.png'
        plt.imsave(fName, mask/255.0)
        image = img.imread(fName)
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)#converting to HSV image
        yellow_mask = cv.inRange(hsv, (61,41,133), (120,255,255))
        green_mask = cv.inRange(hsv, (60, 0, 0), (92, 255,255))
        blue_mask = cv.inRange(hsv, (100, 0, 0), (110, 255,255))
        percentage = lambda mask: round(((mask > 0).mean()) * 100,3)
        greenery_percentage = percentage(yellow_mask)
        land_percentage = percentage(green_mask) + percentage(blue_mask)
        fnfinal=filename.replace('.png','.tif')
        data.append([fnfinal,greenery_percentage,land_percentage])
with open('/content/satelliteImgs/coverPercentage.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)